In [30]:
pip install pyproj

Note: you may need to restart the kernel to use updated packages.


In [31]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [28]:
pip install geopy

  Obtaining dependency information for geopy from https://files.pythonhosted.org/packages/e1/58/9289c6a03116025cdb61461d99b2493daa4967a80b13755463d71a0affeb/geopy-2.4.0-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/40.3 kB ? eta -:--:--
     ---------------------------------------- 40.3/40.3 kB ? eta 0:00:00
   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   ---------------------------------------- 125.4/125.4 kB 7.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
from geopy.geocoders import Nominatim
import pyproj
import geopandas as gpd
import numpy as np
import pandas as pd
from tqdm import tqdm as tq

In [2]:
#도로명 주소->위도, 경도 값으로 바꿔주기
geo_local = Nominatim(user_agent='South Korea')
#위도 경도 반환하는 함수
def geocoding(address):
    try:
        geo=geo_local.geocode(address)
        xy=[geo.latitude, geo.longitude]
        return xy
    except:
        return [0,0]

In [3]:
 # 좌표 변환 함수 정의
def convert_coordinates(lon,lat):
    tm_x, tm_y = transformer.transform(lat, lon)  # 주의: transform 함수는 위도 먼저 받습니다.
    return tm_y, tm_x

In [4]:
# 프로젝션 정의
# 원래 좌표계 (WGS84)
wgs84 = pyproj.CRS('epsg:4326')
# 변환하려는 좌표계 (TM Korea , epsg:5178(새주소지도에서 사용), epsg:5179(네이버지도에서 사용))
tm_korea = pyproj.CRS('epsg:5178')

# Transformer 생성
transformer = pyproj.Transformer.from_crs(wgs84, tm_korea)
 
# 새로운 좌표를 계산하고 새로운 컬럼에 저장
df['x'], df['y'] = zip(*df.apply(lambda row: convert_coordinates(row['경도'], row['위도']), axis=1))

NameError: name 'df' is not defined

In [9]:
#치안센터 UTMK 변환
s_po_xy=pd.read_csv('./data/s_po_xy.csv')
s_po_xy.tail()
lat=[]
lon=[]
for i in tq(s_po_xy['주소']):
    lat.append(geocoding(i)[0])
    lon.append(geocoding(i)[1])

,연번,시도청,경찰서,관서명,구분,전화번호,주소
238,239,서울청,서울수서,대치,지구대,02-538-1312,서울특별시 강남구 삼성로 203
239,240,서울청,서울수서,수서,파출소,02-445-0112,서울특별시 강남구 밤고개로1길 48
240,241,서울청,서울수서,대왕,파출소,02-3411-0323,서울특별시 강남구 헌릉로 623
241,242,서울청,서울수서,일원,지구대,02-3411-0112,서울특별시 강남구 일원로3길 35
242,243,서울청,서울수서,개포,지구대,02-571-0112,서울특별시 강남구 개포로 261


0      중랑구
1       중구
2      종로구
3      은평구
4      용산구
5     영등포구
6      양천구
7      송파구
8      성북구
9      성동구
10     서초구
11    서대문구
12     마포구
13     동작구
14    동대문구
15     도봉구
16     노원구
17     금천구
18     구로구
19     광진구
20     관악구
21     강서구
22     강북구
23     강동구
24     강남구
Name: 자치구, dtype: object

In [27]:
emergency_bell=pd.read_csv('./emergency_bell.csv')
# 프로젝션 정의
# 원래 좌표계 (WGS84)
wgs84 = pyproj.CRS('epsg:4326')
# 변환하려는 좌표계 (TM Korea , epsg:5178(새주소지도에서 사용), epsg:5179(네이버지도에서 사용))
tm_korea = pyproj.CRS('epsg:5178')

# Transformer 생성
transformer = pyproj.Transformer.from_crs(wgs84, tm_korea)
 
# 새로운 좌표를 계산하고 새로운 컬럼에 저장
emergency_bell['x'], emergency_bell['y'] = zip(*emergency_bell.apply(lambda row: convert_coordinates(row['경도'], row['위도']), axis=1))
emergency_bell['관리기관명'].str.contains(gu_list[23])

0         True
1         True
2         True
3         True
4         True
         ...  
20891    False
20892    False
20893    False
20894    False
20895    False
Name: 관리기관명, Length: 20896, dtype: bool

In [30]:
bell_example=pd.DataFrame()
for i in range(len(gu_list)):
    mask=emergency_bell['관리기관명'].str.contains(gu_list[i])
    bell_example=pd.concat([bell_example,emergency_bell[mask].reset_index(drop=True)[0:100]])
bell_example[['x', 'y']]

Index(['번호', '안전비상벨관리번호', '설치목적', '설치장소유형', '설치위치', '소재지도로명주소', '소재지지번주소',
       '위도', '경도', '연계방식', '경찰연계유무', '경비업체연계유무', '관리사무소연계유무', '부가기능',
       '안전비상벨설치년도', '최종점검일자', '최종점검결과구분', '관리기관명', '관리기관전화번호', '데이터기준일자',
       '경찰서명', '경찰서부서명', '경찰서부서코드', '시도경찰청명', '행정동', 'x', 'y'],
      dtype='object')

In [5]:
cctv=pd.read_csv('./seoul_cctv.csv',encoding='CP949')

# 원래 좌표계 (WGS84)
wgs84 = pyproj.CRS('epsg:4326')
# 변환하려는 좌표계 (TM Korea , epsg:5178(새주소지도에서 사용), epsg:5179(네이버지도에서 사용))
tm_korea = pyproj.CRS('epsg:5178')

# Transformer 생성
transformer = pyproj.Transformer.from_crs(wgs84, tm_korea)
 
# 새로운 좌표를 계산하고 새로운 컬럼에 저장
cctv['x'], cctv['y'] = zip(*cctv.apply(lambda row: convert_coordinates(row['경도'], row['위도']), axis=1))


In [9]:
cctv=cctv.drop_duplicates(subset=['x'])

In [11]:
cctv.head()

,자치구,안심 주소,위도,경도,CCTV 수량,수정 일시,x,y
0,중랑구,중화2동 307-1(제일프라자앞)_G-JN00-0007,37.6048,127.0773,1,2022-12-01,962878.984864,1.955931e+06
1,중랑구,신내1동 817-49(고정2)_C-SN01-0026-B,37.6166,127.1090,1,2022-12-01,965682.503313,1.957228e+06
2,중랑구,중화2동 307-1(제일프라자앞사거리)_C-JH02-0061,37.6051,127.0772,1,2022-12-01,962870.307474,1.955965e+06
3,중랑구,신내1동 817 (새솔초교 앞 사거리)_C-SN01-0036,37.6172,127.1090,1,2022-12-01,965682.779609,1.957295e+06
4,중랑구,중화2동 307-4 (제일플라자뒤)_C-JH02-0005,37.6044,127.0781,1,2022-12-01,962949.400293,1.955887e+06


In [26]:
gu_list=cctv['자치구'].drop_duplicates()
gu_list=gu_list.reset_index(drop=True)

cctv_example=pd.DataFrame()
for i in range(len(gu_list)):
    mask=cctv['자치구']==gu_list[i]
    cctv_example=pd.concat([cctv_example,cctv[mask].reset_index(drop=True)[0:100]])
cctv_example.head(3)

,자치구,안심 주소,위도,경도,CCTV 수량,수정 일시,x,y
0,중랑구,중화2동 307-1(제일프라자앞)_G-JN00-0007,37.6048,127.0773,1,2022-12-01,962878.984864,1.955931e+06
1,중랑구,신내1동 817-49(고정2)_C-SN01-0026-B,37.6166,127.1090,1,2022-12-01,965682.503313,1.957228e+06
2,중랑구,중화2동 307-1(제일프라자앞사거리)_C-JH02-0061,37.6051,127.0772,1,2022-12-01,962870.307474,1.955965e+06


In [16]:
cctv_example_utmk=cctv_example[['x', 'y']]
cctv_example_utmk.to_csv('cctv_example_utmk.csv',index=False)

,x,y
0,962878.984864,1.955931e+06
1,965682.503313,1.957228e+06
2,962870.307474,1.955965e+06
3,965682.779609,1.957295e+06
4,962949.400293,1.955887e+06
...,...,...
95,958234.629272,1.946900e+06
96,957970.098484,1.947012e+06
97,961554.861074,1.944487e+06
98,961590.319530,1.944510e+06


In [17]:
bell_example_utmk=bell_example[['x', 'y']]
bell_example_utmk.to_csv('bell_example_utmk.csv',index=False)